In [6]:
# Ethan Mason, Matthew Little
# em45486, ?

In [7]:
import pandas as pd
import numpy as np
import sklearn as sk
import re
import warnings
import torch
from torch.utils.data import TensorDataset, DataLoader
# Suppress FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

test_df.head(10)


,Id,Intake Time,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color,Date of Birth
0,1,1/3/19 16:19,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor,1/3/17
1,2,10/21/13 7:59,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico,9/21/13
2,3,6/29/14 10:38,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,6/29/10
3,4,7/11/15 18:19,Galilee Court And Damita Jo Dr in Manor (TX),Stray,Normal,Dog,Intact Female,5 months,Pit Bull,Brown/White,1/11/15
4,5,2/4/17 10:10,208 Beaver St in Austin (TX),Stray,Injured,Cat,Intact Female,2 years,Domestic Shorthair Mix,Black/White,2/4/15
5,6,3/30/22 9:27,512 Bowery Trl in Austin (TX),Stray,Injured,Dog,Neutered Male,4 years,Beagle,Tricolor,9/3/17
6,7,4/2/21 11:16,Austin (TX),Owner Surrender,Normal,Cat,Intact Female,1 month,Domestic Shorthair Mix,Tortie,2/2/21
7,8,2/21/23 15:26,Ih 35 And Hwy 71 in Austin (TX),Stray,Normal,Dog,Intact Male,2 years,Siberian Husky,Gray/White,10/28/20
8,9,6/29/15 17:04,5306 Peppertree in Austin (TX),Stray,Normal,Cat,Unknown,4 days,Domestic Shorthair Mix,Black,6/25/15
9,10,5/5/15 7:29,4434 Frontier Trl in Austin (TX),Stray,Nursing,Cat,Unknown,2 weeks,Domestic Shorthair Mix,Gray Tabby,4/20/15


## Data Preprocessing

### 🧼 Preprocessing Summary

The following preprocessing steps were applied to prepare the dataset for modeling:

- **Age Conversion**: Converted `Age upon Intake` strings to months using regex, filled missing values with the mean, and renamed the column.

- **Length of Stay**: Calculated `Length of Stay (Days)` as the difference between `Outcome Time` and `Intake Time`, then dropped `Outcome Time`.

- **Datetime Features**: Extracted `Intake Hour`, `Intake DayOfWeek`, and `Intake Month` from `Intake Time`, then dropped the original column.

- **Location Simplification**: Created a boolean feature `Found In Austin` from the `Found Location` string, and dropped the original column.

- **Sex Encoding**: Extracted two features from `Sex upon Intake`: `Sex` (e.g., Intact, Spayed) and `Sex Type` (Male/Female), then dropped the original column.

- **Color Binning**: Kept the top 5 most frequent colors, grouped all others as `"Other"`, one-hot encoded the result, and dropped the original `Color` column.


In [8]:
# Ignore name because that won't affect outcome
train_df = pd.concat([train_df.iloc[:, :1], train_df.iloc[:, 2:]], axis=1)

# Drop ID because that won't affect outcome
train_df.drop('Id', axis=1, inplace=True)

# Drop Date of Birth column since we already have age
train_df.drop('Date of Birth', axis=1, inplace=True)

# Fill the 'Sex upon Intake' NaNs with the mode
train_df['Sex upon Intake'].fillna(train_df['Sex upon Intake'].mode()[0], inplace=True)

# Function to convert age to months
def convert_to_months(age):
    if pd.isna(age):
        return np.nan
    
    # Extract the number and unit using regex
    match = re.match(r'(\d+)\s*(years?|months?|weeks?)', age, re.IGNORECASE)

    if match:
        value = float(match.group(1))
        unit = match.group(2).lower()

        if 'year' in unit:
            return value * 12
        elif 'week' in unit:
            return value / 4.345 # Approx. weeks per month
        else:
            return value
        
    return np.nan

# Calculte how long the animal was with the shelter
def calculate_length_of_stay(df):
    df["Intake Time"] = pd.to_datetime(df["Intake Time"])
    df["Outcome Time"] = pd.to_datetime(df["Outcome Time"])

    df["Length of Stay (Days)"] = (df["Outcome Time"] - df["Intake Time"]).dt.total_seconds() / (60 * 60 * 24)

    df.drop('Outcome Time', axis=1, inplace=True)
    
    return df

# Function to Parse Intake Time
def process_intake_time(df):
    df['Intake Time'] = pd.to_datetime(df['Intake Time'])
    df['Intake Hour'] = df['Intake Time'].dt.hour
    df['Intake DayOfWeek'] = df['Intake Time'].dt.dayofweek
    df['Intake Month'] = df['Intake Time'].dt.month
    df.drop('Intake Time', axis=1, inplace=True)

    return df

def assign_quadrant(location):
    loc = location.lower()
    
    if "austin" not in loc:
        return "Outside Austin"
    
    # Rough heuristic-based quadrant rules
    north = any(word in loc for word in ["north", "n ", "braker", "research"])
    south = any(word in loc for word in ["south", "s ", "slaughter", "william cannon"])
    east  = any(word in loc for word in ["east", "e ", "airport", "springdale"])
    west  = any(word in loc for word in ["west", "w ", "mopac", "bee caves"])

    # Decision logic (priority-based)
    if north and east:
        return "NE"
    elif north and west:
        return "NW"
    elif south and east:
        return "SE"
    elif south and west:
        return "SW"
    elif north:
        return "North"
    elif south:
        return "South"
    elif east:
        return "East"
    elif west:
        return "West"
    else:
        return "Central Austin"


# One-hot Encode the 'Sex upon Intake' column
def encode_sex_upon_intake(df):
    df['Sex'] = df['Sex upon Intake'].str.extract(r'(Intact|Spayed|Neutered)')
    df['Sex Type'] = df['Sex upon Intake'].str.extract(r'(Male|Female)')
    df.drop('Sex upon Intake', axis=1, inplace=True)
    return df

# Bin the top n colors in order to reduce cardinality of the 'Color' column
def bin_top_colors(df, top_n=5):
    # Get the top N most common colors
    top_colors = df["Color"].value_counts().nlargest(top_n).index

    # Create a new column with binned color values
    df["Color Binned"] = df["Color"].apply(lambda c: c if c in top_colors else "Other")

    # Drop the old 'Color' column
    df.drop('Color', axis=1, inplace=True)

    # One-hot encode the new binned color values
    color_dummies = pd.get_dummies(df["Color Binned"], prefix="Color")
    df = pd.concat([df, color_dummies], axis=1)
    
    return df


def preprocess(df):
    df['Age upon Intake'] = df['Age upon Intake'].apply(convert_to_months)
    df["Location Quadrant"] = df["Found Location"].apply(assign_quadrant)
    df = pd.get_dummies(df, columns=["Location Quadrant"])
    df.drop("Found Location", axis=1, inplace=True)

    
    # Compute and apply mean age, if any NaNs remain
    if df['Age upon Intake'].isna().any():
        mean_age = df['Age upon Intake'].mean()
        df['Age upon Intake'].fillna(mean_age, inplace=True)
    
    df.rename(columns={'Age upon Intake': 'Age upon Intake (Months)'}, inplace=True)

    # Only calculate length of stay if Outcome Time is available
    if "Outcome Time" in df.columns:
        df = calculate_length_of_stay(df)

    df = process_intake_time(df)
    df = encode_sex_upon_intake(df)
    df = bin_top_colors(df, top_n=5)

    return df




train_df = preprocess(train_df)


train_df.head(10)

C:\Users\ethan\AppData\Local\Temp\ipykernel_6628\1686473429.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Outcome Time"] = pd.to_datetime(df["Outcome Time"])


,Intake Type,Intake Condition,Animal Type,Age upon Intake (Months),Breed,Outcome Type,Location Quadrant_NE,Location Quadrant_NW,Location Quadrant_North,Location Quadrant_Outside Austin,...,Intake Month,Sex,Sex Type,Color Binned,Color_Black,Color_Black/White,Color_Brown Tabby,Color_Brown Tabby/White,Color_Other,Color_White
0,Stray,Normal,Dog,96.000000,English Springer Spaniel,Return to Owner,False,False,True,False,...,7,Spayed,Female,Other,False,False,False,False,True,False
1,Stray,Normal,Dog,11.000000,Basenji Mix,Return to Owner,False,False,True,False,...,4,Intact,Male,Other,False,False,False,False,True,False
2,Public Assist,Normal,Cat,24.000000,Domestic Shorthair,Transfer,True,False,False,False,...,5,Neutered,Male,Other,False,False,False,False,True,False
3,Owner Surrender,Normal,Dog,24.000000,Labrador Retriever Mix,Return to Owner,False,False,True,False,...,2,Neutered,Male,Other,False,False,False,False,True,False
4,Public Assist,Normal,Dog,72.000000,Great Dane Mix,Return to Owner,True,False,False,False,...,4,Neutered,Male,Black,True,False,False,False,False,False
5,Stray,Normal,Cat,6.000000,Domestic Shorthair,Adoption,True,False,False,False,...,10,Intact,Female,Brown Tabby,False,False,True,False,False,False
6,Public Assist,Normal,Dog,24.000000,Labrador Retriever Mix,Return to Owner,False,False,True,False,...,7,Intact,Male,Black/White,False,True,False,False,False,False
7,Stray,Normal,Cat,0.920598,Domestic Shorthair,Adoption,False,False,False,True,...,6,Intact,Male,Other,False,False,False,False,True,False
8,Stray,Injured,Cat,0.920598,Domestic Shorthair Mix,Transfer,False,False,True,False,...,6,Intact,Female,Black/White,False,True,False,False,False,False
9,Owner Surrender,Normal,Cat,5.000000,Domestic Shorthair Mix,Transfer,False,False,True,False,...,8,Neutered,Male,Other,False,False,False,False,True,False


In [9]:
def refine_features(df):
    import numpy as np
    import pandas as pd

    # --- Simplify Breed ---
    df["Is Mix"] = df["Breed"].str.contains("Mix", case=False, na=False)
    df["Is Domestic"] = df["Breed"].str.contains("Domestic", case=False, na=False)
    df["Is Purebred"] = ~(df["Is Mix"] | df["Is Domestic"])

    # Drop original breed column if not binning top N
    df.drop("Breed", axis=1, inplace=True, errors='ignore')

    # --- Normalize Color (e.g., Black/White == White/Black) ---
    if "Color Binned" not in df.columns and "Color" in df.columns:
        df["Color Normalized"] = df["Color"].fillna("Unknown").apply(
            lambda c: "/".join(sorted(c.split("/"))) if "/" in c else c
        )
        top_colors = df["Color Normalized"].value_counts().nlargest(5).index
        df["Color Binned"] = df["Color Normalized"].apply(lambda x: x if x in top_colors else "Other")
        df.drop(["Color", "Color Normalized"], axis=1, inplace=True)

    # --- Age Group Binning ---
    def age_bin(months):
        if pd.isnull(months):
            return "Unknown"
        if months < 6:
            return "Puppy/Kitten"
        elif months < 24:
            return "Young"
        elif months < 84:
            return "Adult"
        else:
            return "Senior"

    df["Age Group"] = df["Age upon Intake (Months)"].apply(age_bin)

    # --- Intake Time-Based Features ---
    df["Is Weekend"] = df["Intake DayOfWeek"] >= 5
    df["Is Night Intake"] = (df["Intake Hour"] < 6) | (df["Intake Hour"] >= 20)
    df["Is Summer Intake"] = df["Intake Month"].isin([6, 7, 8])

    return df


## Training the Model

Use XGBoost

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import torch

# Preprocess test data
test_df = preprocess(test_df)
train_df = refine_features(train_df)
test_df = refine_features(test_df)

# Separate full train features and labels
X = train_df.drop(columns=["Outcome Type"])
y = train_df["Outcome Type"]

# One-hot encode features
categorical_cols = [
    "Intake Type", "Intake Condition", "Animal Type",
    "Color Binned", "Age Group", "Sex", "Sex Type"
]

X = pd.get_dummies(X, columns=categorical_cols)
X_test = pd.get_dummies(test_df, columns=categorical_cols)

# Align feature columns
X, X_test = X.align(X_test, join="left", axis=1, fill_value=0)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

from xgboost import XGBClassifier

model = XGBClassifier(
    objective='multi:softprob',
    eval_metric='mlogloss',
    use_label_encoder=False,
    num_class=len(label_encoder.classes_),
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=True
)


C:\Users\ethan\AppData\Local\Temp\ipykernel_6628\1686473429.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Intake Time'] = pd.to_datetime(df['Intake Time'])


[0]	validation_0-mlogloss:1.55965
[1]	validation_0-mlogloss:1.50958
[2]	validation_0-mlogloss:1.45937
[3]	validation_0-mlogloss:1.41359
[4]	validation_0-mlogloss:1.36719
[5]	validation_0-mlogloss:1.32416
[6]	validation_0-mlogloss:1.28506


c:\Users\ethan\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [18:16:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[7]	validation_0-mlogloss:1.25113
[8]	validation_0-mlogloss:1.21744
[9]	validation_0-mlogloss:1.18822
[10]	validation_0-mlogloss:1.16049
[11]	validation_0-mlogloss:1.13268
[12]	validation_0-mlogloss:1.11251
[13]	validation_0-mlogloss:1.08746
[14]	validation_0-mlogloss:1.06468
[15]	validation_0-mlogloss:1.04300
[16]	validation_0-mlogloss:1.02466
[17]	validation_0-mlogloss:1.00902
[18]	validation_0-mlogloss:0.99320
[19]	validation_0-mlogloss:0.97546
[20]	validation_0-mlogloss:0.96028
[21]	validation_0-mlogloss:0.94519
[22]	validation_0-mlogloss:0.93126
[23]	validation_0-mlogloss:0.91691
[24]	validation_0-mlogloss:0.90336
[25]	validation_0-mlogloss:0.89261
[26]	validation_0-mlogloss:0.88222
[27]	validation_0-mlogloss:0.87036
[28]	validation_0-mlogloss:0.85931
[29]	validation_0-mlogloss:0.84929
[30]	validation_0-mlogloss:0.84145
[31]	validation_0-mlogloss:0.83176
[32]	validation_0-mlogloss:0.82275
[33]	validation_0-mlogloss:0.81524
[34]	validation_0-mlogloss:0.80693
[35]	validation_0-mlogl

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None, num_class=5, ...)

Make predictions

In [11]:
from sklearn.metrics import classification_report

# Predict on validation set
y_val_pred = model.predict(X_val)
print("\nValidation Set Classification Report:")
print(classification_report(y_val, y_val_pred, target_names=label_encoder.classes_))

# Predict on test set
y_test_pred = model.predict(X_test)
predicted_labels = label_encoder.inverse_transform(y_test_pred)





Validation Set Classification Report:
                 precision    recall  f1-score   support

       Adoption       0.78      0.93      0.85     11009
           Died       0.57      0.06      0.10       208
     Euthanasia       0.66      0.37      0.47       690
Return to Owner       0.84      0.74      0.79      3320
       Transfer       0.79      0.65      0.71      7005

       accuracy                           0.79     22232
      macro avg       0.73      0.55      0.59     22232
   weighted avg       0.79      0.79      0.78     22232



Evaluate Model

### Generate CSV File

In [12]:
# Generate sequential IDs (1-based indexing)
ids = np.arange(1, len(predicted_labels) + 1)

# Create the DataFrame
submission_df = pd.DataFrame({
    'Id': ids,
    'Outcome': predicted_labels
})

# Save to CSV
submission_df.to_csv('submission7.csv', index=False)